## Import dependent libraries

In [2]:
import time
import os
import sys
import math
import random
import numpy as np
from numpy import concatenate, array, asarray

from pandas import read_csv
from pandas import DataFrame
from pandas import concat

from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import mean_squared_error

from matplotlib import pyplot
from IPython.display import SVG
import pydot
import graphviz

import tensorflow as tf

import keras
import keras.backend as K
from keras.utils import np_utils
from keras import optimizers
from keras.models import Model, model_from_json
from keras.layers import Input, Dense, Dropout, BatchNormalization
from keras.layers.recurrent import LSTM
from keras.utils import to_categorical
from keras.models import model_from_json
from keras.layers.core import Activation
from keras.callbacks import ReduceLROnPlateau, LambdaCallback
from keras.utils import plot_model
from sklearn.externals import joblib

from keras.utils.vis_utils import model_to_dot

from NN.ConvolutionalRNN import ConvolutionalRNN
from NN.jan_17_Model import SimpleModel
from Audio.MidiPlayer import MidiPlayer

Using TensorFlow backend.


## Parameters

In [ ]:
# specify the number of lag hours
n_hours = 128
n_features = 2
n_train_hours = None
n_divisions = 4
batch_size = n_hours
epochs = 150
player = MidiPlayer()

## Load dataset

In [ ]:
dataset = read_csv('music_data.csv', header=0)
values = dataset.values
values = values.astype('float32')

## Specify columns to plot

In [ ]:
groups = [0, 1]
i = 1
# plot each column
pyplot.figure()
for group in groups:
    pyplot.subplot(len(groups), 1, i)
    pyplot.plot(values[:, group])
    pyplot.title(dataset.columns[group], y=0.5, loc='right')
    i += 1


pyplot.show()

## Convert series to supervised learning

In [ ]:
dataset = read_csv('music_data.csv', header=1)
n_train_hours = int(len(dataset) * 1)
values = dataset.values
# values = values.astype('float32')

In [ ]:
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
    start = time.time()
    n_vars = 1 if type(data) is list else data.shape[1]
    df = DataFrame(data)
    cols, names = list(), list()
    # input sequence (t-n, ... t-1)
    for i in range(n_in, 0, -1):
        cols.append(df.shift(i))
        names += [('var%d(t-%d)' % (j + 1, i)) for j in range(n_vars)]
    # forecast sequence (t, t+1, ... t+n)
    for i in range(0, n_out):
        cols.append(df.shift(-i))
        if i == 0:
            names += [('var%d(t)' % (j + 1)) for j in range(n_vars)]
        else:
            names += [('var%d(t+%d)' % (j + 1, i)) for j in range(n_vars)]
    # put it all together
    agg = concat(cols, axis=1)
    agg.columns = names
    # drop rows with NaN values
    if dropnan:
        agg.dropna(inplace=True)
    end = time.time()
    print(end - start)
    return agg

## Scale Data

In [ ]:
# scaler = MinMaxScaler(feature_range=(0, 1))
# scaler.fit(values)
# scaled = scaler.transform(values)
# print(scaled[20:21])

In [ ]:
# frame as supervised learning
reframed = series_to_supervised(values, n_hours, 1)

In [ ]:
values = reframed.values
train = values[:, :]
test = values[:, :]
# split into input and outputs
n_obs = n_hours * n_features

In [ ]:
le = LabelEncoder()
ohe = OneHotEncoder(
#     categorical_features = [0]
)

In [ ]:
train_X = train[:, :n_obs]
test_X = test[:, :n_obs]

train_y_volume = test[:, -n_features -1]
train_y_notes = test[:, -n_features]

scaler = MinMaxScaler(feature_range=(0, 1))
train_y_volume = scaler.fit_transform(train_y_volume.reshape(-1,1))


print(train_y_volume[0:10])
# print(train_y_notes[0:20])

test_y_notes = le.fit_transform(train_y_notes)
# test_y_notes = ohe.fit_transform(train_y_notes.reshape(-1, 1)).toarray()
test_y_notes = np_utils.to_categorical(test_y_notes)
# print(test_y_notes[0:10])

# reshape input to be 3D [samples, timesteps, features]
train_X = train_X.reshape((train_X.shape[0], n_hours, n_features))
# print(train_X[0:10])

test_X = test_X.reshape((test_X.shape[0], n_hours, n_features))
# print(train_X.shape, train_y_notes.shape, test_X.shape, test_y_notes.shape)
print(train_y_notes.shape)
print(test_y_notes.shape)
# test_y_notes = test_y_notes.reshape(n_hours, 45)
print(train_X[2])
print(test_y_notes[12])
print(train_y_volume[12])

In [ ]:
visible = Input(name='input', shape=(n_hours, n_features))
conv_rnn = ConvolutionalRNN(visible, n_hours, n_features, n_divisions)
simple = SimpleModel(visible, 128)

In [ ]:
model = conv_rnn.model()
# model = simple.model()

output_notes = Dense(45, activation='softmax', name='output_notes')(model)
output_volume = Dense(1, activation='sigmoid', name='output_volume')(model)

model = Model(inputs=[visible], outputs=[
                                         output_notes, 
                                         output_volume, 
                                        ])

In [ ]:
optimizer = optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
model.compile(loss={'output_notes':'categorical_crossentropy', 'output_volume':'mae'}, optimizer=optimizer)
# model.summary()

In [ ]:
plot_model(model, to_file='model.png')

In [ ]:
SVG(model_to_dot(model).create(prog='dot', format='svg'))

In [ ]:
test_cache = dataset.values[30:n_hours + 30].tolist()
test_data = test_cache.copy()
array_to_play = test_cache.copy()

In [ ]:
print(test_data)

In [ ]:
def process_data(testx):
    testx = DataFrame(data = testx)
    testx = testx.values
    testx = testx.astype('float32')
    testx = scaler.transform(testx)
    return(testx)

In [ ]:
def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds[0], 1)
    return np.argmax(probas)

In [ ]:
def on_epoch_end(epoch, logs):
    global test_cache
    global test_data
    global array_to_play
    if epoch % 10 == 0:
        start = time.time()
        print('----- Generating sound after: %d' % epoch)
        for i in range(400):
            if i == 0:
                to_add = [100, 80]
                array_to_play.append(to_add)
            if i == 100 or i == 500:  
                rand = random.randint(10, 15)
                for i in range(rand):
                    to_add = [0, 0]
                    array_to_play.append(to_add)
                    test_data.append(to_add)
                for i in range(rand):
                    test_data.pop(0)

            if i == 300:    
                for i in range(15):
                    rand = random.randint(45, 65)
                    to_add = [rand, 45]
                    array_to_play.append(to_add)
                    test_data.append(to_add)
                for i in range(15):
                    test_data.pop(0)

            data = process_data(test_data)
            data = data.reshape(1,n_hours,2)
            prediction = model.predict(data)

#             note_prediction = [np.argmax(y, axis=None, out=None) for y in prediction[0]][0]
            note_prediction = sample(prediction[0], temperature = 3)
            
            note_prediction = int(le.inverse_transform(note_prediction))

            volume_prediction = prediction[1][0][0]
            volume_prediction = int(scaler.inverse_transform(volume_prediction)[0][0])
            prediction = [note_prediction, volume_prediction]
            test_data.append(prediction)
            test_data.pop(0)
            array_to_play.append(prediction)
        end = time.time()
        print('time:', end - start)
        print('len array_to_play', len(array_to_play))
        print('array_to_play', array_to_play)
        for value in array_to_play:
            player.play(value[0], .03, value[1])
        test_data = test_cache.copy()
        array_to_play = test_cache.copy()

    
play_callback = LambdaCallback(on_epoch_end=on_epoch_end)

In [ ]:
history = model.fit(
            {'input': train_X},
            {
                'output_notes': test_y_notes, 
                'output_volume': train_y_volume, 
            },
            verbose=1,
            shuffle=False,
            epochs=epochs, 
            batch_size=batch_size,
            callbacks=[play_callback]
)

Epoch 1/150
1920/1961 [============================>.] - ETA: 0s - loss: 3.6059 - output_notes_loss: 3.4052 - output_volume_loss: 0.2007----- Generating sound after: 0
time: 33.22340703010559
len array_to_play 558
array_to_play [[76, 55], [76, 66], [57, 48], [57, 0], [57, 51], [81, 57], [57, 40], [57, 0], [57, 40], [80, 53], [80, 0], [80, 64], [80, 48], [69, 47], [69, 0], [81, 58], [80, 43], [80, 21], [80, 21], [84, 40], [84, 61], [84, 61], [84, 60], [83, 55], [53, 38], [53, 0], [69, 27], [53, 2], [50, 13], [50, 0], [57, 15], [57, 14], [57, 14], [57, 7], [57, 4], [69, 7], [69, 0], [69, 5], [57, 2], [57, 0], [57, 0], [57, 0], [72, 2], [72, 0], [72, 7], [72, 4], [72, 3], [72, 3], [64, 3], [76, 15], [64, 0], [64, 0], [64, 0], [77, 11], [77, 0], [77, 17], [76, 8], [76, 0], [76, 0], [67, 3], [67, 0], [67, 0], [67, 0], [65, 0], [65, 14], [65, 14], [65, 13], [65, 8], [65, 8], [76, 7], [64, 10], [64, 0], [76, 0], [0, 0], [74, 1], [74, 0], [74, 7], [74, 9], [53, 11], [53, 0], [53, 18], [72, 21]

In [ ]:
# serialize model to JSON
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("model.h5")
print("Saved model to disk")

In [ ]:
# plot history
pyplot.plot(history.history['loss'], label='train')
# pyplot.plot(history.history['val_loss'], label='test')
pyplot.legend()
pyplot.show()

In [ ]:
joblib.dump(scaler, 'scaler.pkl')
print('Saved scaler to disk.')